# Example of anomaly plot

In [1]:
# Import to be able to import python package from src
import sys

sys.path.insert(0, '../../src')
import pandas as pd
import ontime as on
from ontime.core.plot.anomaly_plot import AnomalyPlot
from ontime.core.time_series import BinaryTimeSeries, TimeSeries

---
## Load data

In [2]:
data_ts = on.generators.random_walk().generate(start=pd.Timestamp('2021-01-01'), end=pd.Timestamp('2022-12-31'))
data_df = data_ts.pd_dataframe()
data_df.rename(columns={'random_walk': 'data'}, inplace=True)
data_ts = TimeSeries.from_dataframe(data_df)

---
## Creating mock anomalies

In [3]:
# Assuming 'anomalies' is a new column in data_df indicating anomalies with values 0 or 1
import numpy as np

np.random.seed(42)  # Setting seed for reproducibility
anomalies_df = pd.DataFrame({'anomalies': np.random.choice([0, 1], size=len(data_df), p=[0.99, 0.01])},
                            index=data_df.index)

anomalies_ts = BinaryTimeSeries.from_dataframe(anomalies_df)

In [4]:
subset_data_df = anomalies_df.iloc[140:160]
subset_data_df.loc[:, 'anomalies'] = 1
subset_data_ts = BinaryTimeSeries.from_dataframe(subset_data_df)

In [5]:
collective_anomalies_df = anomalies_df.copy()  # Create a copy of anomalies_df

# Initialize the 'anomalies' column in the copy to 0
collective_anomalies_df['anomalies'] = 0

# Set the values to 1 where the specified conditions on the index are met
collective_anomalies_df.loc[(
        ((collective_anomalies_df.index >= '2021-05-21') & (collective_anomalies_df.index <= '2021-05-31')) |
        ((collective_anomalies_df.index >= '2021-06-21') & (collective_anomalies_df.index <= '2021-06-30')) |
        ((collective_anomalies_df.index >= '2021-07-21') & (collective_anomalies_df.index <= '2021-07-31'))
), 'anomalies'] = 1

collective_an_ts = BinaryTimeSeries.from_dataframe(collective_anomalies_df)

---
## Plotting

In [6]:
chart = AnomalyPlot.plot_anomalies(data_ts,
                                   point_anomalies=anomalies_ts,
                                   contextual_anomalies=subset_data_ts,
                                   collective_anomalies=collective_an_ts
                                   )
chart

alt.LayerChart(...)